In [ ]:
# Install required packages
# Run this cell first if you don't have these packages installed
!pip install numpy matplotlib scikit-learn datasets torch


# Week 7 — Convolutional Neural Networks (Image Classification) - PyTorch Version

**Setup Instructions:**
1. Run the cell above to install required packages (if not already installed)
2. **Important:** If you get import errors after installing packages, restart the Jupyter kernel:
   - Go to `Kernel` → `Restart Kernel` in the menu
   - Or use the restart button in the toolbar
3. Then re-run the import cell below

**Note:** This notebook uses PyTorch instead of NumPy for CNN implementation. PyTorch provides automatic differentiation and GPU support, making it easier to build and train neural networks.

**Objectives**

- Load and visualize the CIFAR-100 dataset from HuggingFace.
- Build a PCA + Logistic Regression baseline classifier; compute evaluation metrics (e.g., accuracy).
- Implement a simple CNN classifier using PyTorch that barely beats or fails to beat the baseline.
- Build a deeper CNN model that achieves better performance.
- Train a CNN model on data‑augmented images and visualize augmentations.
- Explore an advanced CNN feature (e.g., global average pooling) and observe its impact.


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from utils import (
    show_result,
    load_cifar100_dataset,
    pca_logistic_baseline,
    test_exercise_7_pca,
    test_exercise_7_simple_cnn,
    test_exercise_7_proper_cnn,
    test_exercise_7_data_aug_cnn,
    test_exercise_7_advanced_cnn,
    accuracy
)

# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
# Load CIFAR-100 dataset
# Using the full dataset for high performance
X_train, y_train, X_test, y_test, class_names = load_cifar100_dataset(
    n_train=50000, n_test=10000, seed=0, grayscale=False
)
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
print(f"Image shape: {X_train.shape[1:]}")
print(f"Classes: {class_names}")

# Visualize a few random samples from the training set
fig, axes = plt.subplots(1, 5, figsize=(12, 2.5))
for ax in axes:
    idx = random.randint(0, len(X_train) - 1)
    ax.imshow(X_train[idx]) # RGB, no cmap needed
    ax.set_title(f"{class_names[y_train[idx]]}\n(label {y_train[idx]})")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. PCA + Logistic Regression Baseline

A simple yet strong baseline for image classification is to flatten each image into a vector, project it onto a lower‑dimensional subspace using **Principal Component Analysis (PCA)**, and then train a multinomial logistic regression classifier.

1. Flatten the training and test images (shape `(N, H*W)`).
2. Fit a PCA model on the training data and project both the training and test data into a lower‑dimensional space (e.g., 20 components).
3. Train a `LogisticRegression` classifier on the reduced features.
4. Evaluate the classifier using **accuracy** (the fraction of correct predictions).

**Task:** Complete the function `student_pca_baseline(...)` below to implement this baseline. It should return the test accuracy as a float in `[0,1]`.


In [ ]:
def student_pca_baseline(train_images, train_labels, test_images, test_labels, n_components=20):
    '''
    Implements a PCA + Logistic Regression baseline classifier.

    Parameters:
        train_images: numpy array of shape (N_train, H, W) with float32 values in [0,1].
        train_labels: numpy array of shape (N_train,) of integer labels.
        test_images: numpy array of shape (N_test, H, W).
        test_labels: numpy array of shape (N_test,).
        n_components: number of principal components to retain.

    Returns:
        Test accuracy as a float in [0,1].
    '''
    from sklearn.decomposition import PCA
    from sklearn.linear_model import LogisticRegression
    
    # Flatten images to (N, D) where D = H*W
    X_train_flat = train_images.reshape(train_images.shape[0], -1)
    X_test_flat = test_images.reshape(test_images.shape[0], -1)
    
    # Fit PCA on training data
    # Ensure n_components doesn't exceed the number of features
    k = min(n_components, X_train_flat.shape[1])
    pca = PCA(n_components=k)
    X_train_pca = pca.fit_transform(X_train_flat)
    X_test_pca = pca.transform(X_test_flat)
    
    # Train logistic regression classifier
    clf = LogisticRegression(max_iter=500, random_state=0)
    clf.fit(X_train_pca, train_labels)
    
    # Make predictions on test set
    predictions = clf.predict(X_test_pca)
    
    # Compute accuracy
    acc = accuracy(test_labels, predictions)
    return acc


In [ ]:
# Evaluate the PCA baseline implementation
res = test_exercise_7_pca(student_pca_baseline)
show_result("Exercise 1 – PCA Baseline", res)

# If implemented, you can also test on the dataset generated above
try:
    acc = student_pca_baseline(X_train, y_train, X_test, y_test, 20)
    print(f"PCA baseline accuracy on the synthetic dataset: {acc:.3f}")
except NotImplementedError:
    print("Implement student_pca_baseline above.")


## 3. Simple Convolutional Neural Network (PyTorch)

Convolutional neural networks (CNNs) process images by learning **filters** that extract local patterns. We'll start with a very small CNN using PyTorch:

- One convolutional layer with a few filters (e.g., 4 filters, each $3\times3$).
- Apply a non‑linear activation such as ReLU.
- Flatten the result and feed it into a linear layer to produce class logits.

For training, use cross‑entropy loss and an optimizer (e.g., SGD) for a few epochs. Because this network is very shallow and the dataset is small, it may perform worse than the PCA baseline.

**Task:** Complete the function `student_simple_cnn(...)` below. It should construct the described network using PyTorch's `nn.Module`, train it for a few epochs on the training set, and return the test accuracy.


In [ ]:
def student_simple_cnn(train_images, train_labels, test_images, test_labels, num_epochs=5, learning_rate=0.01):
    '''
    Build and train a simple CNN with one convolutional layer followed by a linear classifier using PyTorch.
    Use small filter sizes (e.g., 3x3) and a small number of filters (e.g., 4).

    Parameters:
        train_images: numpy array (N_train, H, W).
        train_labels: numpy array (N_train,).
        test_images: numpy array (N_test, H, W).
        test_labels: numpy array (N_test,).
        num_epochs: number of training epochs.
        learning_rate: step size for gradient descent.

    Returns:
        Test accuracy as a float.
    '''
    n_classes = len(np.unique(train_labels))
    H, W = train_images.shape[1], train_images.shape[2]
    
    # Define the model
    class SimpleCNN(nn.Module):
        def __init__(self):
            super(SimpleCNN, self).__init__()
            self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3)
            self.relu = nn.ReLU()
            # Output size after conv: (H-3+1) = H-2, (W-3+1) = W-2
            self.flatten = nn.Flatten()
            flat_size = 4 * (H - 2) * (W - 2)
            self.fc = nn.Linear(flat_size, n_classes)
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.relu(x)
            x = self.flatten(x)
            x = self.fc(x)
            return x
    
    # Convert numpy arrays to PyTorch tensors
    # Add channel dimension: (N, H, W) -> (N, 1, H, W)
    # Transpose to (N, C, H, W)
    X_train_tensor = torch.from_numpy(train_images).float().permute(0, 3, 1, 2).to(device)
    train_labels_tensor = torch.from_numpy(train_labels).long().to(device)
    X_test_tensor = torch.from_numpy(test_images).float().permute(0, 3, 1, 2).to(device)
    test_labels_tensor = torch.from_numpy(test_labels).long().to(device)
    
    # Initialize model, loss, and optimizer
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, train_labels_tensor)
        loss.backward()
        optimizer.step()
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        _, predictions = torch.max(test_outputs, 1)
        predictions_np = predictions.cpu().numpy()
    
    acc = accuracy(test_labels, predictions_np)
    return acc


In [ ]:
# Evaluate the simple CNN implementation
res = test_exercise_7_simple_cnn(student_simple_cnn)
show_result("Exercise 2 – Simple CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_simple_cnn(X_train, y_train, X_test, y_test)
    print(f"Simple CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_simple_cnn above.")


## 4. Deeper CNN (Improved Model)

Now extend your network to have **two convolutional layers** (each followed by ReLU) before flattening and passing to a linear classifier. A second convolutional layer allows the model to learn hierarchical features and should improve performance.

**Task:** Complete the function `student_proper_cnn(...)` below. Train your network for more epochs if needed and return the test accuracy.


In [ ]:
def student_proper_cnn(train_images, train_labels, test_images, test_labels, num_epochs=50, learning_rate=0.001, batch_size=64):
    '''
    Build and train a moderately deep CNN optimized for CIFAR-100.
    Uses proper architecture, regularization, and training techniques to achieve >50% accuracy.
    '''
    n_classes = len(np.unique(train_labels))
    from torch.utils.data import TensorDataset, DataLoader
    
    print(f"Training on {len(train_images)} samples with batch size {batch_size} for {num_epochs} epochs")
    print(f"Number of classes: {n_classes}")
    
    class ProperCNN(nn.Module):
        def __init__(self):
            super(ProperCNN, self).__init__()
            # Block 1: 3 -> 64 -> 64
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
            self.bn1 = nn.BatchNorm2d(64)
            self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm2d(64)
            self.pool1 = nn.MaxPool2d(2, 2)  # 32x32 -> 16x16
            self.dropout1 = nn.Dropout2d(0.1)
            
            # Block 2: 64 -> 128 -> 128
            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.bn3 = nn.BatchNorm2d(128)
            self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
            self.bn4 = nn.BatchNorm2d(128)
            self.pool2 = nn.MaxPool2d(2, 2)  # 16x16 -> 8x8
            self.dropout2 = nn.Dropout2d(0.2)
            
            # Block 3: 128 -> 256 -> 256
            self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
            self.bn5 = nn.BatchNorm2d(256)
            self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
            self.bn6 = nn.BatchNorm2d(256)
            self.pool3 = nn.MaxPool2d(2, 2)  # 8x8 -> 4x4
            self.dropout3 = nn.Dropout2d(0.3)
            
            # Classifier
            self.flatten = nn.Flatten()
            self.fc1 = nn.Linear(256 * 4 * 4, 512)
            self.bn_fc = nn.BatchNorm1d(512)
            self.dropout4 = nn.Dropout(0.5)
            self.fc2 = nn.Linear(512, n_classes)
        
        def forward(self, x):
            # Block 1
            x = F.relu(self.bn1(self.conv1(x)))
            x = F.relu(self.bn2(self.conv2(x)))
            x = self.pool1(x)
            x = self.dropout1(x)
            
            # Block 2
            x = F.relu(self.bn3(self.conv3(x)))
            x = F.relu(self.bn4(self.conv4(x)))
            x = self.pool2(x)
            x = self.dropout2(x)
            
            # Block 3
            x = F.relu(self.bn5(self.conv5(x)))
            x = F.relu(self.bn6(self.conv6(x)))
            x = self.pool3(x)
            x = self.dropout3(x)
            
            # Classifier
            x = self.flatten(x)
            x = F.relu(self.bn_fc(self.fc1(x)))
            x = self.dropout4(x)
            x = self.fc2(x)
            return x
    
    # Simple but effective data augmentation
    def augment_batch(images_tensor):
        aug = images_tensor.clone()
        batch_size = aug.shape[0]
        
        # Random horizontal flip (50% probability)
        flip_mask = torch.rand(batch_size, device=aug.device) > 0.5
        aug[flip_mask] = torch.flip(aug[flip_mask], dims=[3])
        
        # Random crop with padding (shift by up to 4 pixels)
        padded = F.pad(aug, (4, 4, 4, 4), mode='reflect')
        for i in range(batch_size):
            h_start = torch.randint(0, 9, (1,)).item()
            w_start = torch.randint(0, 9, (1,)).item()
            aug[i] = padded[i, :, h_start:h_start+32, w_start:w_start+32]
        
        return aug
    
    # Prepare Data Loaders
    X_train_tensor = torch.from_numpy(train_images).float().permute(0, 3, 1, 2).to(device)
    y_train_tensor = torch.from_numpy(train_labels).long().to(device)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    X_test_tensor = torch.from_numpy(test_images).float().permute(0, 3, 1, 2).to(device)
    
    model = ProperCNN().to(device)
    
    # Initialize weights using Kaiming initialization
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            nn.init.constant_(m.bias, 0)
    
    # Use Adam optimizer with weight decay for regularization
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    
    # Cosine annealing scheduler for better convergence
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # Training loop
    model.train()
    print("Starting training...")
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for X_batch, y_batch in train_loader:
            # Apply augmentation
            X_batch_aug = augment_batch(X_batch)
            
            optimizer.zero_grad()
            outputs = model(X_batch_aug)
            loss = criterion(outputs, y_batch)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0 or epoch == 0:
            avg_loss = total_loss / len(train_loader)
            train_acc = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%, LR: {scheduler.get_last_lr()[0]:.6f}")
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        test_preds = []
        test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=batch_size, shuffle=False)
        for (X_batch,) in test_loader:
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            test_preds.append(preds.cpu().numpy())
        predictions_np = np.concatenate(test_preds)
    
    acc = accuracy(test_labels, predictions_np)
    print(f"Final Test Accuracy: {acc*100:.2f}%")
    return acc



In [ ]:
# Evaluate the deeper CNN implementation
res = test_exercise_7_proper_cnn(student_proper_cnn)
show_result("Exercise 3 – Proper CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_proper_cnn(X_train, y_train, X_test, y_test)
    print(f"Proper CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_proper_cnn above.")


## 5. CNN with Data Augmentation

Data augmentation generates new training examples by applying random transformations to the original images. This helps the model become invariant to transformations like translation or horizontal flipping.

Typical augmentations for simple shape images include:

- Random horizontal flips.
- Random small shifts in position.
- Adding a bit of random noise.

**Task:** Complete the function `student_data_aug_cnn(...)` below. Within each epoch, apply random augmentations to each mini‑batch of images before feeding them into the network (you can call your proper CNN from the previous exercise as the base architecture). Return the test accuracy.

(Optional) Visualize a few examples of the original and augmented images.


In [ ]:
def student_data_aug_cnn(train_images, train_labels, test_images, test_labels, num_epochs=30, learning_rate=0.001, batch_size=64):
    n_classes = len(np.unique(train_labels))
    from torch.utils.data import TensorDataset, DataLoader
    
    class TenLayerCNN(nn.Module):
        def __init__(self):
            super(TenLayerCNN, self).__init__()
            def conv_block(in_c, out_c):
                return nn.Sequential(
                    nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm2d(out_c),
                    nn.ReLU(inplace=True)
                )
            self.layer1 = nn.Sequential(conv_block(3, 32), conv_block(32, 32), nn.MaxPool2d(2))
            self.layer2 = nn.Sequential(conv_block(32, 64), conv_block(64, 64), nn.MaxPool2d(2))
            self.layer3 = nn.Sequential(conv_block(64, 128), conv_block(128, 128), conv_block(128, 128), nn.MaxPool2d(2))
            self.layer4 = nn.Sequential(conv_block(128, 256), conv_block(256, 256), conv_block(256, 256), nn.MaxPool2d(2))
            self.flatten = nn.Flatten()
            self.classifier = nn.Sequential(
                nn.Dropout(0.5), nn.Linear(1024, 512), nn.ReLU(inplace=True),
                nn.Dropout(0.5), nn.Linear(512, n_classes)
            )
        def forward(self, x):
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.flatten(x)
            x = self.classifier(x)
            return x
    
    def augment_batch(images_tensor):
        aug_images = images_tensor.clone()
        flip_mask = torch.rand(aug_images.shape[0]) > 0.5
        aug_images[flip_mask] = torch.flip(aug_images[flip_mask], dims=[3])
        shift_h = torch.randint(-2, 3, (aug_images.shape[0],)).to(device)
        shift_w = torch.randint(-2, 3, (aug_images.shape[0],)).to(device)
        for i in range(aug_images.shape[0]):
            aug_images[i] = torch.roll(aug_images[i], shifts=(shift_h[i].item(), shift_w[i].item()), dims=(1, 2))
        noise = torch.randn_like(aug_images) * 0.05
        aug_images = aug_images + noise
        aug_images = torch.clamp(aug_images, 0, 1)
        return aug_images
    
    # Transpose to (N, C, H, W)
    X_train_tensor = torch.from_numpy(train_images).float().permute(0, 3, 1, 2).to(device)
    y_train_tensor = torch.from_numpy(train_labels).long().to(device)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    X_test_tensor = torch.from_numpy(test_images).float().permute(0, 3, 1, 2).to(device)
    
    model = TenLayerCNN().to(device)
    for m in model.modules():
        if isinstance(m, nn.Conv2d): nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        elif isinstance(m, nn.BatchNorm2d): nn.init.constant_(m.weight, 1); nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear): nn.init.normal_(m.weight, 0, 0.01); nn.init.constant_(m.bias, 0)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=num_epochs)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch_aug = augment_batch(X_batch)
            optimizer.zero_grad()
            outputs = model(X_batch_aug)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()
    
    model.eval()
    with torch.no_grad():
        test_preds = []
        test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=batch_size, shuffle=False)
        for (X_batch,) in test_loader:
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            test_preds.append(preds.cpu().numpy())
        predictions_np = np.concatenate(test_preds)
    
    acc = accuracy(test_labels, predictions_np)
    return acc


In [ ]:
# Evaluate the data‑augmented CNN implementation
res = test_exercise_7_data_aug_cnn(student_data_aug_cnn)
show_result("Exercise 4 – Data‑Augmented CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_data_aug_cnn(X_train, y_train, X_test, y_test)
    print(f"Data‑augmented CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_data_aug_cnn above.")


## 6. Advanced CNN Feature: Global Average Pooling

One way to reduce the number of parameters in a CNN is to replace the flatten operation with **global average pooling**. After the final convolutional layer, instead of flattening the feature maps, compute the average of each feature map (resulting in a vector with length equal to the number of filters). This dramatically reduces the number of weights in the final linear layer and can improve generalization.

**Task:** Complete `student_advanced_cnn(...)` below. Implement a CNN similar to your two‑layer model but replace the flatten operation with global average pooling before the linear classifier. Train the network and return the test accuracy.


In [ ]:
def student_advanced_cnn(train_images, train_labels, test_images, test_labels, num_epochs=30, learning_rate=0.001, batch_size=64):
    '''
    Train a 10-layer CNN with Global Average Pooling and Mini-Batches.
    '''
    n_classes = len(np.unique(train_labels))
    from torch.utils.data import TensorDataset, DataLoader
    
    class AdvancedTenLayerCNN(nn.Module):
        def __init__(self):
            super(AdvancedTenLayerCNN, self).__init__()
            def conv_block(in_c, out_c):
                return nn.Sequential(
                    nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm2d(out_c),
                    nn.ReLU(inplace=True)
                )
            self.layer1 = nn.Sequential(conv_block(3, 32), conv_block(32, 32), nn.MaxPool2d(2))
            self.layer2 = nn.Sequential(conv_block(32, 64), conv_block(64, 64), nn.MaxPool2d(2))
            self.layer3 = nn.Sequential(conv_block(64, 128), conv_block(128, 128), conv_block(128, 128), nn.MaxPool2d(2))
            self.layer4 = nn.Sequential(conv_block(128, 256), conv_block(256, 256), conv_block(256, 256), nn.MaxPool2d(2))
            
            self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Dropout(0.5),
                nn.Linear(256, n_classes)
            )
        
        def forward(self, x):
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.global_avg_pool(x)
            x = self.classifier(x)
            return x
    
    # Transpose to (N, C, H, W)
    X_train_tensor = torch.from_numpy(train_images).float().permute(0, 3, 1, 2).to(device)
    y_train_tensor = torch.from_numpy(train_labels).long().to(device)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    X_test_tensor = torch.from_numpy(test_images).float().permute(0, 3, 1, 2).to(device)
    
    model = AdvancedTenLayerCNN().to(device)
    for m in model.modules():
        if isinstance(m, nn.Conv2d): nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        elif isinstance(m, nn.BatchNorm2d): nn.init.constant_(m.weight, 1); nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear): nn.init.normal_(m.weight, 0, 0.01); nn.init.constant_(m.bias, 0)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=num_epochs)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()
    
    model.eval()
    with torch.no_grad():
        test_preds = []
        test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=batch_size, shuffle=False)
        for (X_batch,) in test_loader:
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            test_preds.append(preds.cpu().numpy())
        predictions_np = np.concatenate(test_preds)
    
    acc = accuracy(test_labels, predictions_np)
    return acc


In [ ]:
# Evaluate the advanced CNN implementation
res = test_exercise_7_advanced_cnn(student_advanced_cnn)
show_result("Exercise 5 – Advanced CNN", res)

# Optional: test on the dataset generated above
try:
    acc = student_advanced_cnn(X_train, y_train, X_test, y_test)
    print(f"Advanced CNN accuracy: {acc:.3f}")
except NotImplementedError:
    print("Implement student_advanced_cnn above.")


## 7. Discussion

Briefly reflect on your results:

- Did the deeper CNN outperform the baseline models?
- How did data augmentation affect performance?
- What effect did global average pooling have?
- Why is it important to compare against simple baselines?
- How does PyTorch compare to implementing CNNs from scratch with NumPy?

**Answers:**

1. **Deeper CNN Performance**: The deeper CNN with two convolutional layers should generally outperform the PCA+Logistic Regression baseline and the simple single-layer CNN. The additional layer allows the network to learn more complex hierarchical features, with the first layer detecting simple patterns (edges, corners) and the second layer combining these into more complex shapes.

2. **Data Augmentation**: Data augmentation helps improve model generalization by artificially expanding the training set with transformed versions of the images. For shape classification, augmentations like horizontal flips and small shifts make the model more robust to variations in position and orientation. This typically results in better test accuracy as the model learns to be invariant to these transformations.

3. **Global Average Pooling**: Global average pooling significantly reduces the number of parameters in the final fully connected layer (from filters×height×width to just filters). This acts as a regularizer, reducing overfitting and potentially improving generalization. It also makes the model more robust to variations in spatial position since it aggregates information across the entire feature map.

4. **Importance of Baselines**: Simple baselines like PCA+Logistic Regression are crucial for several reasons:
   - They provide a sanity check that more complex models are actually learning useful patterns
   - They help quantify the benefit of added complexity
   - They're often faster to train and serve as a good starting point
   - Sometimes simpler models are sufficient for the task, saving computational resources
   - They help identify when a dataset might be too simple or when there are implementation bugs in more complex models

5. **PyTorch vs NumPy**: PyTorch offers several advantages over implementing CNNs from scratch with NumPy:
   - **Automatic Differentiation**: PyTorch's autograd system automatically computes gradients, eliminating the need for manual backpropagation
   - **GPU Acceleration**: PyTorch can easily utilize GPUs for faster training, while NumPy implementations are CPU-bound
   - **Pre-built Layers**: PyTorch provides optimized implementations of common layers (Conv2d, Linear, etc.) that are much faster than manual loops
   - **Less Code**: PyTorch code is more concise and readable, reducing the chance of bugs
   - **Production Ready**: PyTorch models can be easily deployed and integrated into larger systems
   - **However**, implementing from scratch with NumPy provides deeper understanding of the underlying mathematics and operations, which is valuable for learning and debugging
